`Introduction`<br>
Crypto is notoriously difficult to get correct. Shamir quote "Crypto will not be broken, it will be bypassed." <br>
Best to leave it to the `professionals`. aka `server side encryption`<br>
But there are times, it's unavoidable.  In that case, best to follow a good framework the takes care of the details. <br>

`Basic Crypto Requirements` <br>
Semantic Security <br>
Indistinguish under chosen plaintext attack   e.g. encrypting `midway` twice, must result in two different ciphertexts <br>
Indistinguish under chosen ciphertext attacks e.g. no padding oracle 

`Modern Cipher` <br>
AES - NIST standard Advanced Encryption Standard<br>
Block cipher with 128 block size

`Mode of Operations`<br>
GCM - which supports AEAD i.e. authenticated encryption with (additional) associated data<br>
https://nvlpubs.nist.gov/nistpubs/Legacy/SP/nistspecialpublication800-38d.pdf

`Complexity in getting it right`<br>
IV - random<br>
Count - unique<br>
in the GCM case, the most compatible implemenation uses a 96-bit `random` IV and a 32-bit counter

`Two implications`<br>
1. the `number of times` that a key can be used to encrypt()<br>
2. each time the encrypt() is called, the *max* `size` of the data can be encrypted<br>

`other considerations`<br>
where to store IV, counter, tags etc<br>
different programming languages

_______________________________________________________________________________________________________________________________

https://github.com/google/tink  # used to be `key czar`

In [15]:
!tinkey | sed 1,3d | sed s/:\ .*//

 [add-key | convert-keyset |            
 create-keyset | create-public-keyset      
 | delete-key | destroy-key |              
 disable-key | enable-key |                
 list-keyset | list-key-templates |        
 rotate-keyset | promote-key]              


In [23]:
!tinkey list-key-templates | sed 1d | wc -l

103


In [5]:
!tinkey list-key-templates | sed 1d | sort

AES128_CTR_HMAC_SHA256
AES128_CTR_HMAC_SHA256_1MB
AES128_CTR_HMAC_SHA256_4KB
AES128_CTR_HMAC_SHA256_RAW
AES128_EAX
AES128_EAX_RAW
AES128_GCM
AES128_GCM_HKDF_1MB
AES128_GCM_HKDF_4KB
AES128_GCM_RAW
AES256_CMAC
AES256_CMAC_PRF
AES256_CMAC_RAW
AES256_CTR_HMAC_SHA256
AES256_CTR_HMAC_SHA256_1MB
AES256_CTR_HMAC_SHA256_4KB
AES256_CTR_HMAC_SHA256_RAW
AES256_EAX
AES256_EAX_RAW
AES256_GCM
AES256_GCM_HKDF_1MB
AES256_GCM_HKDF_4KB
AES256_GCM_RAW
AES256_SIV
AES256_SIV_RAW
AES_CMAC
AES_CMAC_PRF
CHACHA20_POLY1305
CHACHA20_POLY1305_RAW
ECDSA_P256
ECDSA_P256_IEEE_P1363
ECDSA_P256_IEEE_P1363_WITHOUT_PREFIX
ECDSA_P256_RAW
ECDSA_P384
ECDSA_P384_IEEE_P1363
ECDSA_P521
ECDSA_P521_IEEE_P1363
ECIES_P256_COMPRESSED_HKDF_HMAC_SHA256_AES128_CTR_HMAC_SHA256
ECIES_P256_COMPRESSED_HKDF_HMAC_SHA256_AES128_CTR_HMAC_SHA256_RAW
ECIES_P256_COMPRESSED_HKDF_HMAC_SHA256_AES128_GCM
ECIES_P256_COMPRESSED_HKDF_HMAC_SHA256_AES128_GCM_RAW
ECIES_P256_HKDF_HMAC_SHA256_AES128_CTR_HMAC_SHA256
ECIES_P256_HKDF_HMAC_SHA256_AES128_CTR_HMA

In [13]:
#key creation with a template AES256_GCM

In [25]:
!tinkey create-keyset --key-template AES256_GCM --out aes.gcm.keyset.json

In [17]:
!ls -l *.json

-rw-r--r-- 1 a248042 a248042 260 Aug 25 18:52 aes.gcm.keyset.json


In [27]:
!jq '.key[].keyData.value = "***redacted***"' <aes.gcm.keyset.json  #!jq 'del(.key[].keyData.value)' <aes.gcm.keyset.json

{
  "primaryKeyId": 64052566,
  "key": [
    {
      "keyData": {
        "typeUrl": "type.googleapis.com/google.crypto.tink.AesGcmKey",
        "value": "***redacted***",
        "keyMaterialType": "SYMMETRIC"
      },
      "status": "ENABLED",
      "keyId": 64052566,
      "outputPrefixType": "TINK"
    }
  ]
}


In [7]:
#python code
!which python
!python -V

/home/a248042/py-env/py3.9/bin/python
Python 3.9.6


In [28]:
import tink
from tink import aead
from tink import cleartext_keyset_handle

In [29]:
plaintext = b'top secret'
associated_data = b'this comes from michael, it has *not* be changed'

In [30]:
# 1. Register all AEAD primitives
aead.register()

In [31]:
# 2. Get a handle to the key material.
with open("aes.gcm.keyset.json", 'r') as f:
  aes_gcm_keyset = f.read()
reader = tink.JsonKeysetReader(aes_gcm_keyset)
keyset_handle = cleartext_keyset_handle.read(reader)  #type.googleapis.com/google.crypto.tink.AesGcmKey

In [33]:
# 3. Get the primitive.
aes256_gcm = keyset_handle.primitive(aead.Aead)

In [34]:
# 4. Use the primitive.
ciphertext = aes256_gcm.encrypt(plaintext, associated_data)

# 5. decrypt
decrypted = aes256_gcm.decrypt(ciphertext, associated_data)
print(decrypted, associated_data)

b'top secret' b'this comes from michael, it has *not* be changed'


_______________________________________________________________________________________________________________________________

In [13]:
#java code
package com.incrediblewus;

import com.google.crypto.tink.Aead;
import com.google.crypto.tink.aead.AeadConfig;
import com.google.crypto.tink.KeysetHandle;
import com.google.crypto.tink.KeyTemplates;

/**
 * Hello Tink!
 *
 */
public class App
{
  public static void main(String args[])
    throws Exception
  {
    System.out.println("Hello, Tink");

    String plaintext = "top secret!";
    String aad       = "it has _not_ been changed";

    // 0. register aead config
    AeadConfig.register();

    // 1. Generate the key material.
    KeysetHandle keysetHandle = KeysetHandle.generateNew(KeyTemplates.get("AES256_GCM"));

    // 2. Get the primitive.
    Aead aead = keysetHandle.getPrimitive(Aead.class);

    // 3. Use the primitive to encrypt a plaintext,
    byte[] ciphertext = aead.encrypt(plaintext.getBytes(), aad.getBytes());

    // ... or to decrypt a ciphertext.
    byte[] decrypted = aead.decrypt(ciphertext, aad.getBytes());
    System.out.println(new String(decrypted));
  }
}

In [ ]:
#compare to something like bouncycastle
#https://www.wiley.com/en-us/Beginning+Cryptography+with+Java-p-9780764596339 

In [ ]:
package chapter2;

import javax.crypto.Cipher;
import javax.crypto.spec.IvParameterSpec;
import javax.crypto.spec.SecretKeySpec;

/**
 * Basic symmetric encryption example with padding and CBC using DES
 */
public class SimpleCBCExample
{   
    public static void main(
        String[]    args)
        throws Exception
    {
        byte[] input = new byte[] { 
                0x00, 0x01, 0x02, 0x03, 0x04, 0x05, 0x06, 0x07, 
                0x08, 0x09, 0x0a, 0x0b, 0x0c, 0x0d, 0x0e, 0x0f,
                0x00, 0x01, 0x02, 0x03, 0x04, 0x05, 0x06, 0x07 };
        byte[] keyBytes = new byte[] { 
                0x01, 0x23, 0x45, 0x67, (byte)0x89, (byte)0xab, (byte)0xcd, (byte)0xef };
        byte[] ivBytes = new byte[] { 
                0x07, 0x06, 0x05, 0x04, 0x03, 0x02, 0x01, 0x00 };
        
        SecretKeySpec   key = new SecretKeySpec(keyBytes, "DES");
        IvParameterSpec ivSpec = new IvParameterSpec(ivBytes);
        Cipher          cipher = Cipher.getInstance("DES/CBC/PKCS7Padding", "BC");
        

        System.out.println("input : " + Utils.toHex(input));
        
        // encryption pass
        cipher.init(Cipher.ENCRYPT_MODE, key, ivSpec);
        byte[] cipherText = new byte[cipher.getOutputSize(input.length)];
        int ctLength = cipher.update(input, 0, input.length, cipherText, 0);
        ctLength += cipher.doFinal(cipherText, ctLength);
        System.out.println("cipher: " + Utils.toHex(cipherText, ctLength) + " bytes: " + ctLength);
        
        // decryption pass
        cipher.init(Cipher.DECRYPT_MODE, key, ivSpec);
        byte[] plainText = new byte[cipher.getOutputSize(ctLength)];
        int ptLength = cipher.update(cipherText, 0, ctLength, plainText, 0);
        ptLength += cipher.doFinal(plainText, ptLength);
        System.out.println("plain : " + Utils.toHex(plainText, ptLength) + " bytes: " + ptLength);
    }
}